*AI-Powered Nutrition Analyzer For Fitness Enthusiasts*
**Team ID PNT2022TMID43392** **--SPRINT1--**

**Importing the libraries**

In [1]:
import numpy as np
import tensorflow 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential 

**Importing Flask libraries**

In [2]:
from flask import Flask, render_template,request
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import requests


**Configure ImageDataGenerator class- Image Data Augmentation**

In [3]:
train = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test=ImageDataGenerator(rescale=1./255)

**Apply Image DataGenerator Functionality To Trainset And Testset**
**Loading the data and performing data augmentation**


In [4]:
x_train = train.flow_from_directory(
    r'C:\Users\Naveena\Desktop\IBM\datasets\Dataset\Nutrition Analysis\Dataset\TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test.flow_from_directory(
    r'C:\Users\Naveena\Desktop\IBM\datasets\Dataset\Nutrition Analysis\Dataset\TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [4]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [5]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [6]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

**CREATING THE MODEL-->**
**Adding CNN layers**

In [7]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())



**Adding Dense Layers**

In [8]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2

In [9]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

**Compiling the model**

In [10]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

**Fitting the model-->TRAIN THE MODEL**

In [11]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
526/526 [==============================] - 9s 17ms/step - loss: 0.1196 - accuracy: 0.9585 - val_loss: 0.0703 - val_accuracy: 0.9687
Epoch 2/20
526/526 [==============================] - 8s 16ms/step - loss: 4.4078e-04 - accuracy: 1.0000 - val_loss: 0.0854 - val_accuracy: 0.9536
Epoch 3/20
526/526 [==============================] - 8s 15ms/step - loss: 1.2333e-04 - accuracy: 1.0000 - val_loss: 0.0857 - val_accuracy: 0.9564
Epoch 4/20
526/526 [==============================] - 8s 16ms/step - loss: 5.1556e-05 - accuracy: 1.0000 - val_loss: 0.0782 - val_accuracy: 0.9630
Epoch 5/20
526/526 [==============================] - 8s 16ms/step - loss: 0.0487 - accuracy: 0.9855 - val_loss: 0.0229 - val_accuracy: 0.9915
Epoch 6/20
526/526 [==============================] - 8s 15ms/step - loss: 1.6710e-04 - accuracy: 1.0000 - val_loss: 0.0178 - val_accuracy: 0.9924
Epoch 7/20
526/526 [==============================

**SAVE the model**

In [12]:
classifier.save('nutrition.h5')

**TESTING the model**

In [13]:
model = load_model("nutrition.h5")

In [14]:
img = image.load_img(r"C:\Users\Naveena\Desktop\IBM\datasets\Dataset\Nutrition Analysis\Dataset\TEST_SET\BANANA\12_100.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict_classes(x)
pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([1], dtype=int64)

In [15]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result=str(index[pred[0]])
result

'BANANA'

In [16]:
if(result=='BANANA'):
    print('BANANA - Calories: 112, Fat: 0 grams, Protein: 1 gram and Fiber: 3 grams')
elif(result=='APPLE'):
    print('APPLE - Calories: 104, Vitamin C: 10% of the Daily Value, Carbs: 28 grams and Fiber: 5 grams')
elif(result=='PINEAPPLE'):
    print('PINEAPPLE - Calories: 83, Fat: 1.7 grams, Protein: 1 gram and Fiber: 2.3 grams')
elif(result=='ORANGE'):
    print('ORANGE - Calories: 66, Protein: 1.3 grams, Carbs: 14.8 grams and Carbs: 14.8 grams')
elif(result=='Watermelon'):
    print('WATERMELON - Calories: 46, Protein: 0.9 grams, Fat: 0.2 grams and Fiber: 0.6 grams')


BANANA - Calories: 112, Fat: 0 grams, Protein: 1 gram and Fiber: 3 grams
